In [15]:
from dotenv import load_dotenv

from tqdm.auto import tqdm
import json
load_dotenv()


True

In [23]:
from openai import OpenAI
client = OpenAI()

In [18]:
from elasticsearch import Elasticsearch

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [11]:
with open('../documents.json') as file_rf:
    file_data = json.load(file_rf)

In [12]:
documents = []
for data in file_data:
    for document in data['documents']:
        document['course'] = data['course']
        documents.append(document)

In [13]:
es_client = Elasticsearch("http://localhost:9200")

In [14]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "faqs"

es_client.indices.create(index=index_name,body=index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faqs'})

In [16]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 557.27it/s]


In [17]:
def elastic_search(query):
    query_format = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
    }
    response = es_client.search(index=index_name, body=query_format)
    results = []
    for hit in response['hits']['hits']:
        results.append(hit['_source'])
    
    return results

In [53]:
def build_prompt(query, search_results):
    prompt_template = """ You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION. Please senf relevant ANSWER

    QUESTION : {question}
    CONTEXT : {context}

    """.strip()
    context = ""
    for result in search_results:
        context = context + f"section: {result['section']}\nquestion: {result['question']}\nasnwer: {result['text']}\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [55]:
def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("mps")
    outputs = model.generate(input_ids)
    print(outputs)
    result = tokenizer.decode(outputs[0])
                    
    return result

In [65]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("mps")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [66]:
def rag(query):
    retrieved_data = elastic_search(query)
    prompt = build_prompt(query, retrieved_data)
    print(prompt)
    return llm(prompt)

In [67]:
query ="Course - Can I still join the course after the start date?"
rag(query)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION. Please senf relevant ANSWER

    QUESTION : Course - Can I still join the course after the start date?
    CONTEXT : section: General course-related questions
question: Course - Can I still join the course after the start date?
asnwer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.
section: General course-related questions
question: Course - Can I follow the course after it finishes?
asnwer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capst

/Users/prachisharma/Desktop/git_repos/llm-zoomcamp/zoomcamp/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


RuntimeError: Placeholder storage has not been allocated on MPS device!

/Users/prachisharma/Desktop/git_repos/llm-zoomcamp/zoomcamp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto", torch_dtype=torch.float16)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
input_text = "translate English to Spanish: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("mps")

outputs = model.generate(input_ids)
result = tokenizer.decode(outputs[0])


<pad> <unk> Cuánto edad te es?</s>
